In [12]:
import torch
from config import get_config
config = get_config()
# print(config)
for k,v in config.items():
    print(k,v)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
device = torch.device(device)

batch_size 8
val_epochs 20
lr 0.0001
src_vocab_size 25
tgt_vocab_size 20
seq_len 1024
d_model 960
model_folder weights
model_basename tmodel_
preload latest
experiment_name runs/tmodel
Using device: cuda


In [13]:
from dataloader import get_ds
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)


In [14]:
encoded = tokenizer_tgt.encode("54-122_167-589 | 1.10.640.10")
# Encoding(num_tokens=28, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
encoded.ids
for i in encoded.ids:
    print(tokenizer_tgt.decode([i]))


5
4
-
1
2
2
_
1
6
7
-
5
8
9
 
|
 
1
.
1
0
.
6
4
0
.
1
0


In [15]:
batch = next(iter(train_dataloader))
# print(batch)
print(batch.keys())
# print(type(batch))
encoder_input = batch['encoder_input'].to(device) # (B, seq_len)
print("encoder_input.shape", encoder_input.shape)
decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
print("decoder_input.shape", decoder_input.shape)
encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
print("encoder_mask.shape", encoder_mask.shape)
decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)
print("decoder_mask.shape", decoder_mask.shape)
label = batch['label'].to(device) # (B, seq_len)
print("label.shape", label.shape)
source = batch['src_text']
print(batch['src_text'])
print(batch['tgt_text'])
# print(decoder_mask)

dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'src_text', 'tgt_text'])
encoder_input.shape torch.Size([8, 1024])
decoder_input.shape torch.Size([8, 1024])
encoder_mask.shape torch.Size([8, 1, 1, 1024])
decoder_mask.shape torch.Size([8, 1, 1024, 1024])
label.shape torch.Size([8, 1024])
['MGRYSREPDNPAKSCKARGSNLRVHFKNTYETAMAIKKLPLRRAVRYLKNVVDKKECIPFRRFNGGVGRCAQAKQFGTTQGRWPKKSAEF', 'ELEPVEPYSNEDLDWTDESSRVSREYENEDERDVELVSTEVDDWESYDTVFIGYPIWWAIAAWPVNDFVEANDFTGKTVIPFCTSATSGMGESGELLAELAGTGDWQEGQRFRSGADDKEIQEWVESLGL', 'MEGNTEKPFGIVRGITFSTMNMKRTNLAETEGNPGDQISTLIFRNVEWMGYSETLKISIAMLHLKR', 'MIAGTGIDIASISRMEKGLSRFGARFSRRLLSPGEETLFPAGTKRAAAFLAGRWAAKEAAVKALGTGFAGGVAPSDVSVLSLPSGQPFIRFAGRARERAEELGVTSVHVSITHDAGFAAAIVLLETGLR', 'MRAVTWQGRRDVRVETVPDPRIEEPTDVIVRVTSTGICGSDLHLYEPLGPFLDPGDILGHEPMGIVEEVGGAVTALKPGDRVVVPFNVSCGDCFMCDQGLQSQCETTQVTEYGTGAALFGYTKLYGQVPGGQAEYLRVPFGNTLPVKVGHGPPDDRYVYLSDVLPTAWQAVEYASVPRDGTVVVLGLGPIGDMAARIALHRGAGRVLGVDLVPDRLNRAAAHGVIPLDWRRYGKDLPEAVA

In [21]:
from model import build_transformer_esmc
from config import latest_weights_file_path, get_weights_file_path
model = build_transformer_esmc(tokenizer_tgt.get_vocab_size(), config['seq_len'], d_model=config['d_model']).to(device)

preload = config['preload']
model_filename = latest_weights_file_path(config)
model_filename = "weights/tmodel_400.pt"
if model_filename:
    print(f'Preloading model {model_filename}')
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
else:
    print('No model to preload, starting from scratch')

Preloading model weights/tmodel_400.pt


In [17]:
# Print the number of trainable parameters in the model and all the numbers
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Total trainable parameters: {total_params:,}")

# # Print all parameter counts by module
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"{name}: {param.numel():,}")


Total trainable parameters: 67,922,708


In [30]:
encoder_output = model.encode(batch['src_text'], encoder_mask) # (B, seq_len, d_model)
print(encoder_output.shape)
print(encoder_output)

torch.Size([1, 1024, 960])
tensor([[[ 0.0062, -0.0049,  0.0013,  ...,  0.0020, -0.0018, -0.0099],
         [-0.0284,  0.0368, -0.0025,  ...,  0.0178, -0.0490,  0.0502],
         [ 0.0160,  0.0379,  0.0275,  ...,  0.0014, -0.0507,  0.0165],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0')


In [ ]:
from model import build_transformer
trans_model = build_transformer(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size(), config['seq_len'], config['seq_len'], d_model=config['d_model']).to(device)
encoder_output = trans_model.encode(encoder_input, encoder_mask) # (B, seq_len, d_model)
print(encoder_output.shape)
print(encoder_output)

torch.Size([1, 1024, 960])
tensor([[[-1.0622, -0.9160, -1.9601,  ...,  0.6036, -1.5966, -0.4590],
         [-0.8933, -2.6225, -0.6266,  ...,  0.6246, -2.2994, -0.6068],
         [-0.2721, -2.5294, -0.6001,  ...,  2.1170, -0.8606,  0.4062],
         ...,
         [-0.7625, -2.8094, -1.2348,  ...,  0.9257, -0.8632, -0.4945],
         [-1.3344, -3.0120, -1.7427,  ...,  1.5122, -1.2151, -0.8662],
         [-1.0707, -2.5780, -1.5908,  ...,  0.8420, -1.5378, -0.5994]]],
       device='cuda:0', grad_fn=<AddBackward0>)


: 

In [23]:
from dataset import causal_mask

batch = next(iter(val_dataloader))
# print(batch)
print(batch.keys())
# print(type(batch))
encoder_input = batch['encoder_input'].to(device) # (B, seq_len)
print("encoder_input.shape", encoder_input.shape)
decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
print("decoder_input.shape", decoder_input.shape)
encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
print("encoder_mask.shape", encoder_mask.shape)
decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)
print("decoder_mask.shape", decoder_mask.shape)
label = batch['label'].to(device) # (B, seq_len)
print("label.shape", label.shape)
source = batch['src_text']
print(batch['src_text'])
print(batch['tgt_text'])



sos_idx = tokenizer_tgt.token_to_id('[SOS]')
eos_idx = tokenizer_tgt.token_to_id('[EOS]')

# Precompute the encoder output and reuse it for every step
encoder_output = model.encode(source, encoder_mask)
# Initialize the decoder input with the sos token
decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(encoder_input).to(device)
i = 0
while True:
    if decoder_input.size(1) == 30:
        break

    # build mask for target
    decoder_mask = causal_mask(decoder_input.size(1)).type_as(encoder_input).to(device)

    # calculate output
    out = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)

    # get next token
    prob = model.project(out[:, -1])
    _, next_word = torch.max(prob, dim=1)
    tgt_token = tokenizer_tgt.decode([next_word.item()])
    print(i,tgt_token)
    decoder_input = torch.cat(
        [decoder_input, torch.empty(1, 1).fill_(next_word.item()).type_as(encoder_input).to(device)], dim=1
    )

    if next_word == eos_idx:
        break
    i += 1

print(decoder_input.squeeze(0))
tokenizer_tgt.decode(decoder_input.squeeze(0).tolist())
# return decoder_input.squeeze(0)

dict_keys(['encoder_input', 'decoder_input', 'encoder_mask', 'decoder_mask', 'label', 'src_text', 'tgt_text'])
encoder_input.shape torch.Size([1, 1024])
decoder_input.shape torch.Size([1, 1024])
encoder_mask.shape torch.Size([1, 1, 1, 1024])
decoder_mask.shape torch.Size([1, 1, 1024, 1024])
label.shape torch.Size([1, 1024])
['MLINVLLTLWFYTAHGQGKPCSYPVIKHGRLYYSYRGYFPARVHQQFVYSCDHHFVPPSQRSWDHLTCTAEGWSPEEPCLRQCIFNYLENGHTPHREEKYLQGETVRVRCYEGYSLQNDQNTMTCTESGWSPPPRCIRVKTCSKSNIRIENGFLSESTFTYPLNKQTEYKCKPGYVTADGKTSGLITCLKNGWSAQPVCIKSCDRPVFEKARVKSDGTWFRLNDRLDYECVDGYENRDGRTTGSIVCGQDGWSDKAACYERECSIPEMDPYLNAYPRKETYKVGDVLKFSCSQGRIMVGADSVQCYHFGWSPKLPTCKVKSCALPPELPNGKRKEIHKEEYAHNEVVEYACNPKFLMKGSHKIQCVDGEWTALPVCIEEERTCGNIPDLDHGDVKPSVPPYHHGDSVEFSCREAFTMIGPRFITCISGEWTQPPQSKIQLCPPPPQVPNACDMTTTVNYQDGEKISILCKENYLIQDAEEIVCKGGRWQSIPRCIEKIGCSQPPQLDHGTINSSSSAEERREIHEQRLYAHGTKLSYTCEEGFEISENNVIICHMGKWSSPPQCVGLPCGLPPYIQNGVISHKKDSYQYGEEVTYDCDEGFGTDGPASIRCLGGEWSRPQDCISTNCVNLPTFEDAVLTDREKDFYRSGEKVAFKCLSYYQLDGSNTIQCIK

'1 - 1 1 3   |   3 . 4 0 . 1 0 . 1 0'